<a href="https://colab.research.google.com/github/MayerT1/Sewanee_Colab/blob/main/Split_Creek_Ob_Field_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import requests
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
import json
import struct
from google.colab import files
import zipfile
import os

# URLs for your datasets (only JSON URLs, skip PBF for simplicity)
query_urls = [
    "https://services1.arcgis.com/RpVCghfSj7XHqAKQ/arcgis/rest/services/Canopy_Gap_Drones_/FeatureServer/0/query?f=json&cacheHint=true&orderByFields=&outFields=ObjectId&outSR=102100&where=1=1",
    "https://services1.arcgis.com/RpVCghfSj7XHqAKQ/arcgis/rest/services/Dendro_Sapflux_trees/FeatureServer/0/query?f=json&cacheHint=true&orderByFields=&outFields=ObjectId,Species&outSR=102100&where=1=1",
    "https://services1.arcgis.com/RpVCghfSj7XHqAKQ/arcgis/rest/services/SplitCreek_WeatherStn/FeatureServer/0/query?f=json&cacheHint=true&orderByFields=&outFields=FID&outSR=102100&where=1=1",
    "https://services1.arcgis.com/RpVCghfSj7XHqAKQ/arcgis/rest/services/Soil_Moisture_Markers_/FeatureServer/0/query?f=json&cacheHint=true&orderByFields=&outFields=ObjectId&outSR=102100&where=1=1",
]

def fetch_json_features(url):
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()

    features = data.get("features", [])
    records = []

    for feat in features:
        geometry = feat.get("geometry")
        if geometry is None:
            print(f"Skipping feature with missing geometry: {feat.get('attributes', {}).get('ObjectId', 'No ID')}")
            continue
        try:
            geom = Point(geometry["x"], geometry["y"])
        except Exception as e:
            print(f"Skipping invalid geometry: {e}")
            continue

        props = feat.get("attributes", {})
        records.append({**props, "geometry": geom})

    return records

def main():
    all_gdfs = []

    for url in query_urls:
        print(f"Fetching data from: {url}")
        try:
            records = fetch_json_features(url)
            if records:
                gdf = gpd.GeoDataFrame(records, crs="EPSG:3857")
                gdf = gdf.to_crs("EPSG:4326")
                print(f"Loaded {len(gdf)} features")
                all_gdfs.append(gdf)
            else:
                print("No features returned.")
        except Exception as e:
            print(f"Error fetching/parsing data for URL {url}: {e}")

    if all_gdfs:
        combined = gpd.GeoDataFrame(pd.concat(all_gdfs, ignore_index=True))
        print(f"Combined data: {combined.shape[0]} features")

        # Save shapefile (shapefile consists of multiple files; save in folder)
        out_folder = "combined_shapefile"
        os.makedirs(out_folder, exist_ok=True)
        shp_path = os.path.join(out_folder, "combined_data.shp")
        combined.to_file(shp_path)

        # Zip the shapefile folder contents to download
        zip_path = "combined_shapefile.zip"
        with zipfile.ZipFile(zip_path, 'w') as zipf:
            for root, _, files_in_dir in os.walk(out_folder):
                for file in files_in_dir:
                    file_path = os.path.join(root, file)
                    zipf.write(file_path, arcname=file)

        print(f"Saved shapefile to {zip_path}")

        # Download the zipped shapefile
        files.download(zip_path)
    else:
        print("No data to save.")

if __name__ == "__main__":
    main()


Fetching data from: https://services1.arcgis.com/RpVCghfSj7XHqAKQ/arcgis/rest/services/Canopy_Gap_Drones_/FeatureServer/0/query?f=json&cacheHint=true&orderByFields=&outFields=ObjectId&outSR=102100&where=1=1
Loaded 4 features
Fetching data from: https://services1.arcgis.com/RpVCghfSj7XHqAKQ/arcgis/rest/services/Dendro_Sapflux_trees/FeatureServer/0/query?f=json&cacheHint=true&orderByFields=&outFields=ObjectId,Species&outSR=102100&where=1=1
Loaded 103 features
Fetching data from: https://services1.arcgis.com/RpVCghfSj7XHqAKQ/arcgis/rest/services/SplitCreek_WeatherStn/FeatureServer/0/query?f=json&cacheHint=true&orderByFields=&outFields=FID&outSR=102100&where=1=1
Loaded 1 features
Fetching data from: https://services1.arcgis.com/RpVCghfSj7XHqAKQ/arcgis/rest/services/Soil_Moisture_Markers_/FeatureServer/0/query?f=json&cacheHint=true&orderByFields=&outFields=ObjectId&outSR=102100&where=1=1
Loaded 9 features
Combined data: 117 features
Saved shapefile to combined_shapefile.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [13]:
import geopandas as gpd
import folium

# Load the shapefile
gdf = gpd.read_file("/content/combined_shapefile/combined_data.shp")

# Ensure the CRS is WGS84 (lat/lon) for Leaflet
if gdf.crs.to_string() != "EPSG:4326":
    gdf = gdf.to_crs(epsg=4326)

# Get the centroid of the geometries to center the map
centroid = gdf.geometry.unary_union.centroid
map_center = [centroid.y, centroid.x]

# Create a folium map centered on the data
m = folium.Map(location=map_center, zoom_start=15)

# Add the GeoDataFrame as a GeoJSON layer to the map
folium.GeoJson(gdf).add_to(m)

# Save to an HTML file and display map inline if using Jupyter
m.save("domain_buildings_map.html")
m


/tmp/ipython-input-1374458856.py:12: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  centroid = gdf.geometry.unary_union.centroid
